In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

preprocessing the dataset into one hot encoded list

In [2]:
rebuild_data = True
if(os.path.exists("Training_set.npy")):
    rebuild_data = False

class Preprocess():
    Img_size = 50
    cats = "PetImages/Cat"
    dogs = "PetImages/Dog"
    Train_set = []
    labels = {cats:0 , dogs:1}
    count_cats = 0
    count_dogs = 0
    def make_train_set(self):
        for label in self.labels:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label , f)
                    img = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img , (self.Img_size , self.Img_size))
                    img_label = np.eye(2)[self.labels[label]]
                    self.Train_set.append([np.array(img) , img_label])
                    if label == self.cats:
                        self.count_cats+=1
                    elif label == self.dogs:
                        self.count_dogs += 1
                except Exception as e:
                    pass

        np.random.shuffle(self.Train_set)
        self.Train_set = np.asarray(self.Train_set , dtype = object)
        np.save("Training_set.npy" , self.Train_set)
        print("Cats:" , self.count_cats)
        print("Dogs:" , self.count_dogs)

if rebuild_data:
    preprocess = Preprocess()
    preprocess.make_train_set()

In [3]:
training_set = np.load("Training_set.npy" , allow_pickle=True)

In [4]:
class Net(nn.Module):
    def __init__(self ):
        super().__init__()
        self.conv1 = nn.Conv2d(1 , 32 , kernel_size=5)
        self.conv2 = nn.Conv2d(32 , 64 , kernel_size=5)
        self.conv3 = nn.Conv2d(64 , 128 , kernel_size=5)

        self.flatten_dim = None
        ##definig a temp x to get the flatten dimensions after a forward pass
        x = torch.randn(50 , 50).view(-1 , 1 , 50 , 50)
        self.convf(x)

        self.fc1 = nn.Linear(self.flatten_dim , 512)
        self.fc2 = nn.Linear(512 , 2)
        

    def convf(self , x ):
        x = F.max_pool2d(F.relu(self.conv1(x)) , (2 , 2))
        x = F.max_pool2d(F.relu(self.conv2(x)) , (2 , 2))
        x = F.max_pool2d(F.relu(self.conv3(x)) , (2 , 2))

        if self.flatten_dim == None:
            self.flatten_dim = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]

        return x

    def forward(self , x):
        x = self.convf(x)
        x = x.view(-1 , self.flatten_dim)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.softmax(x ,dim = 1 )

net = Net()

In [5]:
training_set_x = torch.tensor(np.array([i[0] for i in training_set])).view(-1 , 50 , 50)
training_set_x = training_set_x/255.0
training_set_y = torch.tensor(np.array([i[1] for i in training_set]))

val_percent = 0.1
val_size = int(val_percent*len(training_set_x))

train_set_x = training_set_x[:-val_size]
train_set_y = training_set_y[:-val_size]

val_set_x = training_set_x[-val_size:]
val_set_y = training_set_y[-val_size:]

In [6]:
BATCH_SIZE = 10
EPOCS = 5
LEARNING_RATE = 0.001

optimizer = optim.Adam(net.parameters() , LEARNING_RATE)
loss_function = nn.MSELoss()

def train(net):
    for epoc in range(EPOCS):
        for i in tqdm(range(0 , len(train_set_x) , BATCH_SIZE)):
            x = train_set_x[i:i+BATCH_SIZE].view(-1 , 1 , 50 , 50).float()
            y = train_set_y[i:i+BATCH_SIZE].float()
            net.zero_grad()
            output = net(x)
            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
        print(f"loss after {epoc+1} iterations: {loss}")

In [7]:
def test(net):
    correct = 0
    total = 0   
    with torch.no_grad():
        for i in tqdm(range(len(val_set_x))):
            actual_class = torch.argmax(val_set_y[i])
            predicted_class = torch.argmax(net(val_set_x[i].view(-1 , 1 , 50 , 50)))
            if predicted_class==actual_class:
                correct +=1
            total+=1
    print(f'Accuracy on test set: {round(correct/total*100 , 3)}')

In [8]:
train(net)

  0%|          | 0/2246 [00:00<?, ?it/s]

100%|██████████| 2246/2246 [00:25<00:00, 88.45it/s]


loss after 1 iterations: 0.2704133987426758


100%|██████████| 2246/2246 [00:25<00:00, 87.98it/s]


loss after 2 iterations: 0.3610239028930664


...

loss after 3 iterations: 0.4827135503292084


100%|██████████| 2246/2246 [00:25<00:00, 86.66it/s]


loss after 4 iterations: 0.409868985414505


100%|██████████| 2246/2246 [00:25<00:00, 88.77it/s]

loss after 5 iterations: 0.4454355239868164


In [9]:
test(net)

100%|██████████| 2494/2494 [00:02<00:00, 1154.10it/s]

Accuracy on test set: 73.617


In [10]:
def forw_pass(x , y , train = False):
    if train:
        net.zero_grad()
    try:
        output = net(x.view(-1 , 1, 50 , 50))
        matches = [torch.argmax(i)==torch.argmax(j) for i , j in zip(y , output)]
        accuracy = matches.count(True)/len(matches)*100
        if train:
            loss = loss_function(output , y)
            loss.backward()
            optimizer.step()

        print(f"Accuracy: {accuracy}")
    except Exception as e:
        print("Wrong image dimensions!!")

In [11]:
forw_pass(val_set_x , val_set_y)

Accuracy: 73.61668003207699


<h2>Test on your own image</h2>

In [33]:
def predictCatOrDog(path):
    img = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img , (50 , 50))

    img = torch.tensor(img).view(-1 , 1 , 50 , 50)
    img = img/255.0

    prediction = torch.argmax(net(img))
    if prediction==0:
        print("cat")
    else:
        print("Dog")

predictCatOrDog("Your image")

Dog
